<a href="https://colab.research.google.com/github/klmnjulian/pytorch-cifar10-classifier/blob/main/cifar10_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


100%|██████████| 170498071/170498071 [00:13<00:00, 13107487.26it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

In [7]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [12]:
epochs = 10

In [13]:
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 0.926
[1,  4000] loss: 0.946
[1,  6000] loss: 0.946
[1,  8000] loss: 0.947
[1, 10000] loss: 0.956
[1, 12000] loss: 0.946
[2,  2000] loss: 0.873
[2,  4000] loss: 0.872
[2,  6000] loss: 0.913
[2,  8000] loss: 0.905
[2, 10000] loss: 0.923
[2, 12000] loss: 0.928
[3,  2000] loss: 0.816
[3,  4000] loss: 0.848
[3,  6000] loss: 0.865
[3,  8000] loss: 0.868
[3, 10000] loss: 0.894
[3, 12000] loss: 0.873
[4,  2000] loss: 0.776
[4,  4000] loss: 0.804
[4,  6000] loss: 0.829
[4,  8000] loss: 0.849
[4, 10000] loss: 0.841
[4, 12000] loss: 0.873
[5,  2000] loss: 0.751
[5,  4000] loss: 0.781
[5,  6000] loss: 0.796
[5,  8000] loss: 0.820
[5, 10000] loss: 0.828
[5, 12000] loss: 0.829
[6,  2000] loss: 0.725
[6,  4000] loss: 0.773
[6,  6000] loss: 0.759
[6,  8000] loss: 0.794
[6, 10000] loss: 0.803
[6, 12000] loss: 0.807
[7,  2000] loss: 0.703
[7,  4000] loss: 0.725
[7,  6000] loss: 0.732
[7,  8000] loss: 0.778
[7, 10000] loss: 0.790
[7, 12000] loss: 0.798
[8,  2000] loss: 0.689
[8,  4000] 

In [11]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))


Accuracy of the network on the 10000 test images: 60 %
